In [3]:
import numpy as np
import pandas as pd
import glob
import json
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch #conda install pytorch==1.2.0 torchvision==0.4.0 -c pytorch
from ast import literal_eval
from cdqa.utils.filters import filter_paragraphs #pip install cdqa
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.pipeline.cdqa_sklearn import QAPipeline

download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')

# Loading data and filtering / preprocessing the documents
df = pd.read_csv('data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)

# Loading QAPipeline with CPU version of BERT Reader pretrained on SQuAD 1.1
cdqa_pipeline = QAPipeline()

/Users/pennguyen/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)



bnpp_newsroom-v1.1.csv already downloaded

bert_qa.joblib already downloaded


In [7]:
import numpy as np
import pandas as pd
import glob
import json
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv

In [9]:
root_path = '/Users/pennguyen/Downloads/CORD-19-research-challenge'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

/Users/pennguyen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,8q5ondtn,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(72)90077-4
1,pzfd0e50,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(80)90355-5
2,22bka3gi,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(80)90356-7
3,zp9k1k3z,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(73)90176-9
4,cjuzul89,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-9343(85)90361-4


In [73]:
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

52097

In [74]:
import random

random.shuffle(all_json)

all_json_og = all_json

all_json = all_json[:10000]

In [75]:
all_json

['/Users/pennguyen/Downloads/CORD-19-research-challenge/custom_license/custom_license/pdf_json/586b872fe6b35fee039e9e6535087e3ea26e11f8.json',
 '/Users/pennguyen/Downloads/CORD-19-research-challenge/comm_use_subset/comm_use_subset/pmc_json/PMC4015617.xml.json',
 '/Users/pennguyen/Downloads/CORD-19-research-challenge/comm_use_subset/comm_use_subset/pdf_json/16627f4c7134394da448b1417a771d13ad7cca4a.json',
 '/Users/pennguyen/Downloads/CORD-19-research-challenge/custom_license/custom_license/pmc_json/PMC7097038.xml.json',
 '/Users/pennguyen/Downloads/CORD-19-research-challenge/noncomm_use_subset/noncomm_use_subset/pmc_json/PMC5783036.xml.json',
 '/Users/pennguyen/Downloads/CORD-19-research-challenge/custom_license/custom_license/pdf_json/60b986e136014d085c001e0475523adf1b0eb22d.json',
 '/Users/pennguyen/Downloads/CORD-19-research-challenge/comm_use_subset/comm_use_subset/pdf_json/aa1f783a1af8b14cdbee5ea50cc0a78c56d7442e.json',
 '/Users/pennguyen/Downloads/CORD-19-research-challenge/comm_us

In [86]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            if "abstract" in content:
                for entry in content['abstract']:
                    self.abstract.append(entry['text'])
            # Body text
            self.body_text.append("")
#             for entry in content['body_text']:
#                 self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
#             self.body_text = '\n'.join(self.body_text)
            # Paragraphs
            self.paragraphs = []
            for entry in content['body_text']:
                self.paragraphs.append(entry['text'])
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'

In [87]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [88]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'paragraphs': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    dict_['paragraphs'].append(content.paragraphs)
    
    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0: 
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    
    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(". ".join(authors[:2]) + "...")
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])

    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'paragraphs', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

Processing index: 0 of 10000
Processing index: 1000 of 10000
Processing index: 2000 of 10000
Processing index: 3000 of 10000
Processing index: 4000 of 10000
Processing index: 5000 of 10000
Processing index: 6000 of 10000
Processing index: 7000 of 10000
Processing index: 8000 of 10000
Processing index: 9000 of 10000


,paper_id,abstract,body_text,paragraphs,authors,title,journal,abstract_summary
0,586b872fe6b35fee039e9e6535087e3ea26e11f8,"The ruminant pestiviruses, bovine virus diarrh...",[],[There are three pestiviruses. Hog cholei-a vi...,"Nettleton, P.F.. Entrican, G.",Ruminant pestiviruses,British Veterinary Journal,"The ruminant pestiviruses, bovine virus<br>di..."
1,16627f4c7134394da448b1417a771d13ad7cca4a,Background: Baseline data is necessary for mon...,[],[If an outbreak of pandemic influenza should o...,"Barr, Margo. Raphael, Beverley...",Pandemic influenza in Australia: Using<br>tel...,BMC Infect Dis,Background: Baseline data is necessary for<br...
2,60b986e136014d085c001e0475523adf1b0eb22d,,[],[Bacterial infection of the hepatic parenchyma...,"Overturf, Gary D.",Chapter 10 Focal Bacterial Infections,Infectious Diseases of the Fetus and Newborn I...,Not provided.
3,aa1f783a1af8b14cdbee5ea50cc0a78c56d7442e,Emergence of a new influenza strain leads to a...,[],[Virus-specific CD8 + T cells play a critical ...,"Valkenburg, Sophie A.. Gras, Stephanie...",Protective Efficacy of Cross-Reactive CD8(+)<...,PLoS Pathog,Emergence of a new influenza strain leads to ...
4,2c6d57a5c31e345cd689687255b13506d71df902,The genus Sapovirus belongs to the family Cali...,[],[junctional complexes comprising tight junctio...,"Alfajaro, Mia Madel. Cho, Eun-Hyo...",Early Porcine Sapovirus Infection Disrupts<br...,J Virol,The genus Sapovirus belongs to the family<br>...


In [54]:
df_covid.paragraphs[0][9]

"Leadership's support of the program evidenced through visible leadership presence, messaging and release of resources is the key factor in helping to make the program a true success. The hospital was recognised as a Global Hand Hygiene Expert Centre in January 2011 ( Figure 14) . The use of the WHO multi-prong interventions is successful in improving hand hygiene compliance with concomitant reduction in healthcareassociated infections. "

In [89]:
df_covid = df_covid.drop(columns=['abstract', 'body_text', 'journal', 'abstract_summary'])

In [90]:
df_covid

,paper_id,paragraphs,authors,title
0,586b872fe6b35fee039e9e6535087e3ea26e11f8,[There are three pestiviruses. Hog cholei-a vi...,"Nettleton, P.F.. Entrican, G.",Ruminant pestiviruses
1,16627f4c7134394da448b1417a771d13ad7cca4a,[If an outbreak of pandemic influenza should o...,"Barr, Margo. Raphael, Beverley...",Pandemic influenza in Australia: Using<br>tel...
2,60b986e136014d085c001e0475523adf1b0eb22d,[Bacterial infection of the hepatic parenchyma...,"Overturf, Gary D.",Chapter 10 Focal Bacterial Infections
3,aa1f783a1af8b14cdbee5ea50cc0a78c56d7442e,[Virus-specific CD8 + T cells play a critical ...,"Valkenburg, Sophie A.. Gras, Stephanie...",Protective Efficacy of Cross-Reactive CD8(+)<...
4,2c6d57a5c31e345cd689687255b13506d71df902,[junctional complexes comprising tight junctio...,"Alfajaro, Mia Madel. Cho, Eun-Hyo...",Early Porcine Sapovirus Infection Disrupts<br...
5,c8a09ce3cbd5d88ceb3bb530003b2f4a46d92d3f,"[Historically, infectious diseases were believ...","Holz, Peter H.. Lumsden, Linda F....",Virus survey in populations of two subspecies...
6,7eb8da93320ea097a81497d97ed22bdad6e708dc,[The novel Coronavirus (2019-nCoV) came to wor...,"Escher, Allan R",An Ounce of Prevention: Coronavirus<br>(COVID...
7,4fe40da0ad3fd356b77f5c2fc8d8fbb7d7ad71c1,[Ascorbic acid (vitamin C) is absorbed by the ...,"Wilson, John X.. Jeffrey Dixon, S.",Ascorbic acid transport in mouse and rat<br>a...
8,d2b714f43fa6302301a1aa7d282de555f71024b2,[There is great interest in the development of...,"Wang, Chuan. Hart, Matthew...",Germinal Center B Cell and T Follicular Helpe...
9,75bee23cd59f44b79301f951a5627c47a1adcee3,[Venezuelan equine encephalitis virus (VEEV) i...,"Lundberg, Lindsay. Fontenot, Jacque...",Venezuelan Equine Encephalitis Virus Capsid<b...


In [4]:
df = pd.read_csv('data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)

In [91]:
cdqa_pipeline.fit_retriever(df=df_covid)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...ize=32,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [6]:
cdqa_pipeline.predict(query='your question', n_predictions=3)

[('Paribas Foundation supports the project "',
  'Climate change: stimulating effective adaptation programs in Africa',
  'Today it is no longer a question of whether or not anthropogenic global warming is a reality, but rather a question of how we adapt.  As part of its Climate Initiative program, the BNP Paribas Foundation supports the project "Africa less vulnerable". Its goal ? Modelling extreme climate change events in Africa and their effects to help populations reduce their exposure to these phenomena.',
  3.7301367916062915),
 ('polar expedition is unthinkable without the logistics provided by IP',
  'Antarctica, the indicator of climate change',
  '"Such a mission is of course of scientific interest, but is also of logistical and human interest," says documentary filmmaker Pascal Guérin, accustomed to extreme filming. "A polar expedition is unthinkable without the logistics provided by IPEV and is, in some respects, even more extreme than a stay on the space station. The psych

In [92]:
# Sending a question to the pipeline and getting prediction
query = 'Are smokers and tobacco users at higher risk of COVID-19 infection?'
prediction = cdqa_pipeline.predict(query=query)

print('query: {}\n'.format(query))
print('answer: {}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: Are smokers and tobacco users at higher risk of COVID-19 infection?

answer: janitor and security guard will inevitably face the risk of COVID-

title:  A COVID-19 Infection Risk Model for Frontline<br>Health Care Workers

paragraph: Every doctor, nurse, medical technologist, radiation technologist, nursing assistant, hospital janitor and security guard will inevitably face the risk of COVID-19 infection. Here, we formulate a model to investigate how many frontliners are expected to be infected under certain scenarios [8] . We use this expected number of possible new infections as a measure of the risk.

